In [15]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/fake-news/submit.csv
/kaggle/input/fake-news/train.csv
/kaggle/input/fake-news/test.csv


In [16]:
import pandas as pd

In [17]:
df = pd.read_csv("/kaggle/input/fake-news/train.csv")

In [18]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [19]:
df.shape

(20800, 5)

In [20]:
df.isnull().sum()

id           0
title      558
author    1957
text        39
label        0
dtype: int64

In [21]:
df = df.dropna()

In [22]:
df.isnull().sum()

id        0
title     0
author    0
text      0
label     0
dtype: int64

In [23]:
df.shape

(18285, 5)

In [24]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [25]:
X = df.drop("label",axis=1)

In [26]:
X.head()

,id,title,author,text
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ..."
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...


In [27]:
Y = df["label"]

In [28]:
Y

0        1
1        0
2        1
3        1
4        1
        ..
20795    0
20796    0
20797    0
20798    1
20799    1
Name: label, Length: 18285, dtype: int64

In [29]:
import tensorflow as tf
print(tf.__version__)

2024-06-17 03:57:06.133010: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-06-17 03:57:06.133139: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-06-17 03:57:06.306335: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


2.15.0


In [30]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense

In [31]:
voc_size = 5000

## One hot representation

In [32]:
messages = X.copy()

In [36]:
messages['title'][1]

'FLYNN: Hillary Clinton, Big Woman on Campus - Breitbart'

In [37]:
messages

,id,title,author,text
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ..."
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...
...,...,...,...,...
20795,20795,Rapper T.I.: Trump a ’Poster Child For White S...,Jerome Hudson,Rapper T. I. unloaded on black celebrities who...
20796,20796,"N.F.L. Playoffs: Schedule, Matchups and Odds -...",Benjamin Hoffman,When the Green Bay Packers lost to the Washing...
20797,20797,Macy’s Is Said to Receive Takeover Approach by...,Michael J. de la Merced and Rachel Abrams,The Macy’s of today grew from the union of sev...
20798,20798,"NATO, Russia To Hold Parallel Exercises In Bal...",Alex Ansary,"NATO, Russia To Hold Parallel Exercises In Bal..."


In [38]:
messages.reset_index(inplace=True)

In [39]:
messages

,index,id,title,author,text
0,0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...
1,1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...
2,2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ..."
3,3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...
4,4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...
...,...,...,...,...,...
18280,20795,20795,Rapper T.I.: Trump a ’Poster Child For White S...,Jerome Hudson,Rapper T. I. unloaded on black celebrities who...
18281,20796,20796,"N.F.L. Playoffs: Schedule, Matchups and Odds -...",Benjamin Hoffman,When the Green Bay Packers lost to the Washing...
18282,20797,20797,Macy’s Is Said to Receive Takeover Approach by...,Michael J. de la Merced and Rachel Abrams,The Macy’s of today grew from the union of sev...
18283,20798,20798,"NATO, Russia To Hold Parallel Exercises In Bal...",Alex Ansary,"NATO, Russia To Hold Parallel Exercises In Bal..."


In [40]:
import nltk
import re
from nltk.corpus import stopwords

In [52]:
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()
corpus = []
for i in range(0,len(messages)):
    #print(i)
    review = re.sub('^[a-zA-Z]',' ',messages['title'][i])
    review = review.lower()
    review = review.split(' ')
    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)

In [54]:
messages['title'][0]

'House Dem Aide: We Didn’t Even See Comey’s Letter Until Jason Chaffetz Tweeted It'

In [56]:
corpus

[' ous dem aide: didn’t even see comey’ letter jason chaffetz tweet',
 ' lynn: hillari clinton, big woman campu - breitbart',
 ' hy truth might get fire',
 '15 civilian kill singl us airstrik identifi',
 ' ranian woman jail fiction unpublish stori woman stone death adulteri',
 ' acki mason: hollywood would love trump bomb north korea lack tran bathroom (exclus video) - breitbart',
 ' enoît hamon win french socialist party’ presidenti nomin - new york time',
 '  back-channel plan ukrain russia, courtesi trump associ - new york time',
 ' bama’ organ action partner soros-link ‘indivisible’ disrupt trump’ agenda',
 ' bc comedi sketch "real housew isis" caus outrag',
 ' ussian research discov secret nazi militari base ‘treasur hunter’ arctic [photos]',
 ' offici see link trump russia',
 ' e: yes, paid govern troll social media, blogs, forum websit',
 ' n major leagu soccer, argentin find home success - new york time',
 ' ell fargo chief abruptli step - new york time',
 ' nonym donor pay $2.

In [57]:
onehot_repr = [one_hot(words,voc_size) for words in corpus]

In [58]:
onehot_repr

[[1304, 3321, 46, 1374, 2091, 1207, 1960, 2326, 2023, 625, 426],
 [4700, 2301, 1516, 4128, 4418, 97, 4033],
 [157, 4375, 903, 1933, 4066],
 [345, 2459, 767, 2660, 3974, 4717, 2234],
 [3871, 4418, 2595, 4951, 3914, 3444, 4418, 138, 312, 820],
 [367,
  2963,
  3909,
  2905,
  1343,
  211,
  3673,
  1589,
  1740,
  3474,
  205,
  4454,
  707,
  397,
  4033],
 [2809, 328, 2420, 1039, 213, 1067, 2208, 3018, 3615, 4945, 2318],
 [3149, 2837, 4904, 4801, 367, 2429, 211, 4770, 3615, 4945, 2318],
 [1140, 1742, 1030, 4499, 1205, 4850, 2, 623, 1230, 1760],
 [4192, 4983, 1091, 498, 1259, 1109, 1051, 3451],
 [3315, 3883, 2359, 1466, 3326, 1732, 3500, 1553, 27, 2792, 2467],
 [1371, 1207, 4850, 211, 367],
 [1036, 2730, 3703, 1103, 4094, 2354, 1368, 2930, 2623, 1872],
 [1155, 30, 295, 4662, 843, 2858, 3835, 4747, 3615, 4945, 2318],
 [2296, 4387, 4321, 462, 231, 3615, 4945, 2318],
 [4469, 4225, 4936, 2811, 288, 289, 3763, 4229, 2731, 2170, 1439, 1872],
 [3481, 3278, 953],
 [4366, 2437, 3256, 4333, 211, 

## Embedding Techniques

In [59]:
sent_length = 20
embedded_docs = pad_sequences(onehot_repr,padding="pre",maxlen=sent_length)
print(embedded_docs)

[[   0    0    0 ... 2023  625  426]
 [   0    0    0 ... 4418   97 4033]
 [   0    0    0 ...  903 1933 4066]
 ...
 [   0    0    0 ... 3615 4945 2318]
 [   0    0    0 ...  718 3149 1890]
 [   0    0    0 ... 1980 4296  405]]


In [61]:
#Creating Model
embedding_vector_features = 40
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(LSTM(100))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentrophy',optimizer='adam',metrics=['accuracy'])
print(model.summary())


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_1 (Embedding)         │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_1 (LSTM)                   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

None


In [62]:
## Creating model
embedding_vector_features=40 ##features representation
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(LSTM(100))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_2 (Embedding)         │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_2 (LSTM)                   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

None


In [63]:
voc_size


5000

In [65]:
len(embedded_docs),Y.shape

(18285, (18285,))

In [67]:
import numpy as np
X_final = np.array(embedded_docs)
y_final = np.array(Y)

In [68]:
X_final.shape,y_final.shape

((18285, 20), (18285,))

In [74]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(X_final,y_final,test_size=0.33,random_state=42)

In [70]:
X_final

array([[   0,    0,    0, ..., 2023,  625,  426],
       [   0,    0,    0, ..., 4418,   97, 4033],
       [   0,    0,    0, ...,  903, 1933, 4066],
       ...,
       [   0,    0,    0, ..., 3615, 4945, 2318],
       [   0,    0,    0, ...,  718, 3149, 1890],
       [   0,    0,    0, ..., 1980, 4296,  405]], dtype=int32)

In [76]:
model.fit(x_train,y_train,validation_data=(x_test,y_test),epochs=10,batch_size=64)

Epoch 1/10
192/192 ━━━━━━━━━━━━━━━━━━━━ 8s 27ms/step - accuracy: 0.7851 - loss: 0.4302 - val_accuracy: 0.9185 - val_loss: 0.1913
Epoch 2/10
192/192 ━━━━━━━━━━━━━━━━━━━━ 5s 25ms/step - accuracy: 0.9444 - loss: 0.1347 - val_accuracy: 0.9193 - val_loss: 0.1904
Epoch 3/10
192/192 ━━━━━━━━━━━━━━━━━━━━ 5s 27ms/step - accuracy: 0.9698 - loss: 0.0810 - val_accuracy: 0.9173 - val_loss: 0.2189
Epoch 4/10
192/192 ━━━━━━━━━━━━━━━━━━━━ 5s 25ms/step - accuracy: 0.9828 - loss: 0.0528 - val_accuracy: 0.9137 - val_loss: 0.2505
Epoch 5/10
192/192 ━━━━━━━━━━━━━━━━━━━━ 5s 25ms/step - accuracy: 0.9894 - loss: 0.0313 - val_accuracy: 0.9160 - val_loss: 0.2898
Epoch 6/10
192/192 ━━━━━━━━━━━━━━━━━━━━ 5s 25ms/step - accuracy: 0.9935 - loss: 0.0233 - val_accuracy: 0.9123 - val_loss: 0.3706
Epoch 7/10
192/192 ━━━━━━━━━━━━━━━━━━━━ 5s 25ms/step - accuracy: 0.9952 - loss: 0.0182 - val_accuracy: 0.9102 - val_loss: 0.4348
Epoch 8/10
192/192 ━━━━━━━━━━━━━━━━━━━━ 5s 25ms/step - accuracy: 0.9959 - loss: 0.0138 - val_accu

In [77]:
##Performance Mertric and Accuracy

In [83]:
y_pred = model.predict(x_test)


189/189 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step


In [88]:
y_pred = np.where(y_pred > 0.5,1,0)

In [89]:
y_pred

array([[1],
       [0],
       [0],
       ...,
       [0],
       [1],
       [1]])

In [86]:
y_pred.shape

(6035, 1)

#Performance Metrics and Accuracy

from sklearn.metrics import confustion_matrix

In [91]:
from sklearn.metrics import confusion_matrix

In [92]:
confusion_matrix(y_test,y_pred)

array([[3106,  313],
       [ 226, 2390]])

In [94]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)

0.9106876553438277

In [96]:
from sklearn.metrics import classification_report
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.93      0.91      0.92      3419
           1       0.88      0.91      0.90      2616

    accuracy                           0.91      6035
   macro avg       0.91      0.91      0.91      6035
weighted avg       0.91      0.91      0.91      6035

